In [5]:
import pandas as pd 
!pip install wikipedia
import wikipedia as wp
from bs4 import BeautifulSoup
!pip install lxml
import requests 

In [6]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Chennai").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all('li')[11:]:
    neighborhoodList.append(row.text)
ch_df = pd.DataFrame({"Neighborhood": neighborhoodList})

ch_df

,Neighborhood
0,Red Hills
1,Royapuram
2,Korukkupet
3,Vyasarpadi
4,Tondiarpet
...,...
435,Developers
436,Statistics
437,Cookie statement
438,


In [7]:
ch_df.drop(ch_df.index[162:439], inplace=True)
ch_df

,Neighborhood
0,Red Hills
1,Royapuram
2,Korukkupet
3,Vyasarpadi
4,Tondiarpet
...,...
158,Chembarambakkam
159,Singaperumalkoil
160,Maraimalai nagar
161,Urapakkam


In [4]:
import requests
import os
from sklearn.cluster import KMeans
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geocoder --yes
import geocoder # to get coordinates

from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\admin\Anaconda3_1

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2




geocoder-1.38.1      | 53 KB     |            |   0% 
geocoder-1.38.1      | 53 KB     | ###        |  30% 
geocoder-1.38.1      | 53 KB     | ######     |

In [8]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, Tamil Nadu'.format(neighborhood))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
    return lat_lng_coords

In [10]:
coords = [ get_latlng(neighborhood) for neighborhood in ch_df["Neighborhood"].tolist() ]

NameError: name 'geocoder' is not defined

In [9]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

NameError: name 'coords' is not defined

In [8]:
ch_df['Latitude'] = df_coords['Latitude']
ch_df['Longitude'] = df_coords['Longitude']

In [21]:
print(ch_df.shape)
ch_df
ch_df.drop(ch_df.index[[162]], inplace=True)

(163, 3)


In [22]:
ch_df.to_csv("ch_df.csv", index=False)
ch_df = pd.read_csv('ch_df.csv')
ch_df

,Neighborhood,Latitude,Longitude
0,Red Hills,13.19543,80.18431
1,Royapuram,13.11394,80.29420
2,Korukkupet,13.11680,80.27726
3,Vyasarpadi,13.11778,80.25168
4,Tondiarpet,13.12923,80.28955
...,...,...,...
157,Kattupakkam,13.04152,80.12898
158,Chembarambakkam,13.04027,80.06437
159,Singaperumalkoil,12.76333,80.00350
160,Maraimalai nagar,12.80656,80.04193


In [23]:
from geopy.geocoders import Nominatim 
# get the coordinates of Chennai 
address = 'Chennai, Tamil Nadu'

geolocator = Nominatim(user_agent="Chennai")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, Tamil Nadu {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai, Tamil Nadu 13.0801721, 80.2838331.


In [24]:
!pip install folium
import folium 
# create map of chennai using latitude and longitude values
map_ch = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ch_df['Latitude'], ch_df['Longitude'], ch_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ch)  
    
map_ch

In [25]:
map_ch.save('map_ch.html')

In [1]:
CLIENT_ID = 'T0CIPBIR32HEE3VWZY0XXJB225K1EGABMNDIJL3LWCFXDOPN' # your Foursquare ID
CLIENT_SECRET = 'RTXHJIFTP220PTD1UXLIQSN3UTN1MNAZQIUPZ011XYVGPLIA' # your Foursquare Secret
VERSION = '20201108' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET )

Your credentails:
CLIENT_ID: T0CIPBIR32HEE3VWZY0XXJB225K1EGABMNDIJL3LWCFXDOPN
CLIENT_SECRET:RTXHJIFTP220PTD1UXLIQSN3UTN1MNAZQIUPZ011XYVGPLIA


In [2]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(ch_df['Latitude'], ch_df['Longitude'], ch_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
  # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
       

NameError: name 'ch_df' is not defined

In [ ]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)


In [ ]:
venues_df.head()

In [ ]:
venues_df.groupby(["Neighborhood"]).count()

In [ ]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

In [ ]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

In [ ]:
venues_df['Neighborhood'] == 'Tambaram'

In [ ]:
# one hot encoding
ch_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ch_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ch_onehot.columns[-1]] + list(ch_onehot.columns[:-1])
ch_onehot = ch_onehot[fixed_columns]

print(ch_onehot.shape)
ch_onehot.head(5)

In [ ]:
ch_grouped = ch_onehot.groupby(["Neighborhoods"]).mean().reset_index()

ch_grouped

In [ ]:
ch_grouped.shape

In [ ]:
num_top_venues = 5

for hood in ch_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = ch_grouped[ch_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
import numpy as np 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ch_grouped['Neighborhoods']

for ind in np.arange(ch_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ch_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
# set number of clusters
kclusters = 5

ch_clustering = ch_grouped.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ch_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
kmeans.labels_

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ch_merged = ch_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ch_merged = ch_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ch_merged # check the last columns!

In [ ]:
type(kmeans.labels_)


In [ ]:
ch_merged.drop(ch_merged.index[[23,43,53,87,135]], inplace=True)

In [ ]:
ch_merged.shape

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ch_merged['Latitude'], ch_merged['Longitude'], ch_merged['Neighborhood'], ch_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
map_clusters.save('map_clusters.html')

## Cluster 1

In [ ]:
ch_merged.loc[ch_merged['Cluster Labels'] == 0]

## Cluster 2

In [ ]:
ch_merged.loc[ch_merged['Cluster Labels'] == 1]

## Cluster 3

In [ ]:
ch_merged.loc[ch_merged['Cluster Labels'] == 2]

## Cluster 4

In [ ]:
ch_merged.loc[ch_merged['Cluster Labels'] == 3]

## Cluster 5

In [ ]:
ch_merged.loc[ch_merged['Cluster Labels'] == 4]

## Thank You